In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../../_apikeys.env")
api_key = os.getenv("Doogie.2ndKey")
os.environ['OPENAI_API_KEY'] = api_key

In [3]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings( model="text-embedding-ada-002" )
query_result = embeddings.embed_query( "나는 배가 아주 많이 고픈것 같다. 그렇기 때문에 나는 식사를 할 것인지, 아니면 굶을 것인지 고민이다.")

print(query_result);


[-0.0017008718701964404, -0.04127319934487488, 0.015299771100829617, -0.016488604376631657, -0.01847860790351768, 0.0030819217450004845, -0.01926685698510031, -0.005718031011575204, -0.024332320508082914, -0.010059857819740017, -0.006829331682433632, 0.01971912958261546, -0.00991125319460347, -0.02922979765167694, -0.004981470798949606, 0.015054251185174849, 0.02871291361871953, 0.0010014630466856243, 0.004693954288447693, -0.023298552442168097, -0.012359991766400484, -0.010214923029627239, 0.020946730092211888, -0.018556140508461293, -0.0025165796011226735, 0.015441914209892906, 0.018749972952142903, -0.011461906224798278, -0.011681581938805178, -0.0049943928997735415, 0.04763087295025101, -0.013438987650860365, 0.0005499970327340652, 0.007985860171837125, 0.011720348241276983, 0.0043612099594007156, 0.0034469710932766543, 0.005042851010693944, -0.01550652471401258, -0.015338537403301423, -0.013232234037677402, -0.007927710252468126, 0.009685116895845896, -0.013839572776402357, 0.0175

In [4]:
print(query_result.__sizeof__())
print(len(query_result))

12328
1536


In [6]:
#from numpy import dot
#from numpy.linalg import norm

import pandas as pd

somedata = [ '저는 배가 고파요',
             '저기 배가 지나 가네요',
             '굶어서 허기가 지네요',
             '허기 워리라는 게임이 즐거워요',
             '스팀에서 재미있는 것 해야지',
             '스팀으로 연어구이 해 먹을 거야',
             ]
#'저기 있는 배가 맛있을 것 같아요'
df = pd.DataFrame(somedata, columns=['SampleText'])
df

,SampleText
0,저는 배가 고파요
1,저기 배가 지나 가네요
2,굶어서 허기가 지네요
3,허기 워리라는 게임이 즐거워요
4,스팀에서 재미있는 것 해야지
5,스팀으로 연어구이 해 먹을 거야


In [7]:
def get_embedding(text):
    return embeddings.embed_query(text)

df['EmbedVec'] = df.apply(lambda row: get_embedding(row.SampleText), axis=1)
df


,SampleText,EmbedVec
0,저는 배가 고파요,"[-0.016637360179694062, -0.021788898203996463,..."
1,저기 배가 지나 가네요,"[-0.0052657917147770515, -0.024830033460230574..."
2,굶어서 허기가 지네요,"[-0.006181030746934309, -0.0069507935034026636..."
3,허기 워리라는 게임이 즐거워요,"[-0.013712686737581248, -0.00977461896104763, ..."
4,스팀에서 재미있는 것 해야지,"[-0.007083486745692401, -0.01767247537766439, ..."
5,스팀으로 연어구이 해 먹을 거야,"[-0.0020729016852510007, -0.02858352643788332,..."


In [8]:
for embedVec in df.EmbedVec:
    print( len(embedVec), embedVec[:2] )

1536 [-0.016637360179694062, -0.021788898203996463]
1536 [-0.0052657917147770515, -0.024830033460230574]
1536 [-0.006181030746934309, -0.0069507935034026636]
1536 [-0.013712686737581248, -0.00977461896104763]
1536 [-0.007083486745692401, -0.01767247537766439]
1536 [-0.0020729016852510007, -0.02858352643788332]


In [12]:
def cos_sim(a, b):
    return dot(a, b) / (norm(a) * norm(b))

def calc_sim(df, queryValue):
    queryValue_embedding = get_embedding(queryValue)
    #df['Similarity'] = df.EmbedVec.apply( cos_sim, args=(queryValue_embedding,))
    df['Similarity'] = df.EmbedVec.apply( lambda row: cos_sim(row, queryValue_embedding) )
    #df['Similarity'] = df.EmbedVec.apply( lambda row: cos_sim(np.array(row), np.array(queryValue_embedding) ) )
    #df['Similarity'] = df.apply( lambda row: cos_sim(np.array(row.EmbedVec), np.array(queryValue_embedding) ), axis=1)
    #df['Similarity'] = df.apply( lambda row: cos_sim(row.EmbedVec, queryValue_embedding), axis=1)
    #df['Similarity'] = df.apply( lambda row: cos_sim(row.EmbedVec, queryValue_embedding), axis=1)

calc_sim(df, "아무것도 안 먹었더니 꼬르륵 소리가 나네")
df


          SampleText                                           EmbedVec  \
0          저는 배가 고파요  [-0.016637360179694062, -0.021788898203996463,...   
1       저기 배가 지나 가네요  [-0.0052657917147770515, -0.024830033460230574...   
2        굶어서 허기가 지네요  [-0.006181030746934309, -0.0069507935034026636...   
3   허기 워리라는 게임이 즐거워요  [-0.013712686737581248, -0.00977461896104763, ...   
4    스팀에서 재미있는 것 해야지  [-0.007083486745692401, -0.01767247537766439, ...   
5  스팀으로 연어구이 해 먹을 거야  [-0.0020729016852510007, -0.02858352643788332,...   

   Similarity  
0    0.812184  
1    0.809394  
2    0.836768  
3    0.785841  
4    0.788828  
5    0.826998  


In [10]:
def get_top3(df):
    df_top3 = df.sort_values(by=['Similarity'], ascending=False).head(3)
    return df_top3

result_top3 = get_top3(df)
result_top3

,SampleText,EmbedVec,Similarity
2,굶어서 허기가 지네요,"[-0.006181030746934309, -0.0069507935034026636...",0.836768
5,스팀으로 연어구이 해 먹을 거야,"[-0.0020729016852510007, -0.02858352643788332,...",0.826998
0,저는 배가 고파요,"[-0.016637360179694062, -0.021788898203996463,...",0.812184
